In [4]:
import re
import PyPDF2 as pd
import argparse

In [7]:
parser = argparse.ArgumentParser(  # Задаем новый парсер
    description='Найдите в своем файле регулярные повторы!')
parser.add_argument('pdf', metavar='PDF', type=str,  # Теперь задаем аргументы
                    help='PDF-File for opening')  # (объекты) для парсера,
parser.add_argument('regular', metavar='RE', type=str,
                    help='''Get the RE from your PDF
                    (default: count pages from PDF''')
args = parser.parse_args()

usage: ipykernel_launcher.py [-h] PDF RE
ipykernel_launcher.py: error: the following arguments are required: RE


SystemExit: 2

/home/alexmakh/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
#p = pd.PdfFileReader(open("srep30698.pdf", mode='rb')).getNumPages()
#type(p)

In [5]:
p = pd.PdfFileReader(open(pdf, mode='rb'))
o = ""
for i in range(p.getNumPages()):
    page = p.getPage(i)
    o += page.extractText()
    i += 1
c = re.findall('Ly-6/uPAR', o)
print(c)
p = pd.PdfFileReader(open(pdf, mode='rb')).getNumPages()
print(p)

Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR
Ly-6/uPAR



import sys
import PyPDF2
from PIL import Image

if (len(sys.argv) != 2):
    print("\nUsage: python {} input_file\n".format(sys.argv[0]))
    sys.exit(1)

pdf = sys.argv[1]

if __name__ == '__main__':
    input1 = PyPDF2.PdfFileReader(open("srinivas1992.pdf", "rb"))
    page0 = input1.getPage(30)

if '/XObject' in page0['/Resources']:
    xObject = page0['/Resources']['/XObject'].getObject()

for obj in xObject:
    if xObject[obj]['/Subtype'] == '/Image':
        size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
        data = xObject[obj].getData()
    if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
        mode = "RGB"
    else:
        mode = "P"

    if '/Filter' in xObject[obj]:
        if xObject[obj]['/Filter'] == '/FlateDecode':
            img = Image.frombytes(mode, size, data)
            img.save(obj[1:] + ".png")
        elif xObject[obj]['/Filter'] == '/DCTDecode':
            img = open(obj[1:] + ".jpg", "wb")
            img.write(data)
            img.close()
        elif xObject[obj]['/Filter'] == '/JPXDecode':
            img = open(obj[1:] + ".jp2", "wb")
            img.write(data)
            img.close()
        elif xObject[obj]['/Filter'] == '/CCITTFaxDecode':
            img = open(obj[1:] + ".tiff", "wb")
            img.write(data)
            img.close()
        else:
            img = Image.frombytes(mode, size, data)
            img.save(obj[1:] + ".png")
    else:
        print("No image found.")

#!/usr/bin/env python
"""
Concatenate pages from pdf files into a single pdf file.

Page ranges refer to the previously-named file.
A file not followed by a page range means all the pages of the file.

PAGE RANGES are like Python slices.
        {page_range_help}
EXAMPLES
    pdfcat -o output.pdf head.pdf content.pdf :6 7: tail.pdf -1
        Concatenate all of head.pdf, all but page seven of content.pdf,
        and the last page of tail.pdf, producing output.pdf.

    pdfcat chapter*.pdf >book.pdf
        You can specify the output file by redirection.

    pdfcat chapter?.pdf chapter10.pdf >book.pdf
        In case you don't want chapter 10 before chapter 2.
"""
# Copyright (c) 2014, Steve Witham <switham_github@mac-guyver.com>.
# All rights reserved. This software is available under a BSD license;
# see https://github.com/mstamy2/PyPDF2/LICENSE

from __future__ import print_function
import argparse
from PyPDF2.pagerange import PAGE_RANGE_HELP


def parse_args():
    parser = argparse.ArgumentParser(
        description=__doc__.format(page_range_help=PAGE_RANGE_HELP),
        formatter_class=argparse.RawDescriptionHelpFormatter)
    parser.add_argument("-o", "--output",
                        metavar="output_file")
    parser.add_argument("-v", "--verbose", action="store_true",
                        help="show page ranges as they are being read")
    parser.add_argument("first_filename", nargs=1,
                        metavar="filename [page range...]")
    # argparse chokes on page ranges like "-2:" unless caught like this:
    parser.add_argument("fn_pgrgs", nargs=argparse.REMAINDER,
                        metavar="filenames and/or page ranges")
    args = parser.parse_args()
    args.fn_pgrgs.insert(0, args.first_filename[0])
    return args


from sys import stderr, stdout, exit
import os
import traceback
from collections import defaultdict

from PyPDF2 import PdfFileMerger, parse_filename_page_ranges


if __name__ == "__main__":
    args = parse_args()
    filename_page_ranges = parse_filename_page_ranges(args.fn_pgrgs)
    if args.output:
        output = open(args.output, "wb")
    else:
        stdout.flush()
        output = os.fdopen(stdout.fileno(), "wb")

    merger = PdfFileMerger()
    in_fs = dict()
    try:
        for (filename, page_range) in filename_page_ranges:
            if args.verbose:
                print(filename, page_range, file=stderr)
            if filename not in in_fs:
                in_fs[filename] = open(filename, "rb")
            merger.append(in_fs[filename], pages=page_range)
    except:
        print(traceback.format_exc(), file=stderr)
        print("Error while reading " + filename, file=stderr)
        exit(1)
    merger.write(output)
    # In 3.0, input files must stay open until output is written.
    # Not closing the in_fs because this script exits now.